In [1]:
import os
os.chdir("../")

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


k_fold = 10
seeds = [192760, 1393659, 1269293, 138973, 931308, 1088652, 1361151, 1456105, 820535, 1240839]


In [3]:
data_train_path = "Dataset/data_train.csv"
data_test_path = "Dataset/data_target_users_test.csv"

In [4]:
df = pd.read_csv(data_train_path)
df_test = pd.read_csv(data_test_path)

df.columns = ["UserID", "ItemID", "Interaction"]
df_test.columns = ["UserID"]

In [5]:
df

,UserID,ItemID,Interaction
0,1,7,1.0
1,1,15,1.0
2,1,16,1.0
3,1,133,1.0
4,1,161,1.0
...,...,...,...
478725,13024,13605,1.0
478726,13024,13823,1.0
478727,13024,15122,1.0
478728,13024,18185,1.0


In [6]:
metric_to_optimize="RECALL"
cutoff_to_optimize=20

In [7]:
user_ids = df["UserID"].unique().tolist()
user2user_encoded = {x: i for i, x in enumerate(user_ids)}
userencoded2user = {i: x for i, x in enumerate(user_ids)}
item_ids = df["ItemID"].unique().tolist()
item2item_encoded = {x: i for i, x in enumerate(item_ids)}
item_encoded2item = {i: x for i, x in enumerate(item_ids)}
df["User"] = df["UserID"].map(user2user_encoded)
df["Item"] = df["ItemID"].map(item2item_encoded)

num_users = len(user2user_encoded)
num_items = len(item_encoded2item)
df["Interaction"] = df["Interaction"].values.astype(np.float32)

# min and max ratings will be used to normalize the ratings later
min_rating = 0.0
max_rating = max(df["Interaction"])

print(
    "Number of users: {}, Number of Items: {}, Min rating: {}, Max rating: {}".format(
        num_users, num_items, min_rating, max_rating
    )
)

Number of users: 12638, Number of Items: 22222, Min rating: 0.0, Max rating: 1.0


In [8]:
userId_unique = df["UserID"].unique()
itemId_unique = df["ItemID"].unique()

In [9]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, KFold
import numpy as np
import scipy.sparse as sps
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample


urm_all = sps.coo_matrix((df["Interaction"].values, 
                          (df["User"].values, df["Item"].values)))

urm_train_validation = []
urm_test = []
urm_train = []
urm_validation = []

for i in range(len(seeds)):
    a, b = split_train_in_two_percentage_global_sample(urm_all, train_percentage = 0.80,seed=seeds[i])
    c, d = split_train_in_two_percentage_global_sample(a, train_percentage = 0.80,seed=seeds[i])
    urm_train_validation.append(a)
    urm_test.append(b)
    urm_train.append(c)
    urm_validation.append(d)



In [10]:
from Evaluation.Evaluator import EvaluatorHoldout
evaluator_validation = []
for u in urm_validation:
    evaluator_validation.append(EvaluatorHoldout(u, cutoff_list=[20], ignore_users=[]))

EvaluatorHoldout: Ignoring 2581 (20.4%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Ignoring 2598 (20.6%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Ignoring 2621 (20.7%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Ignoring 2601 (20.6%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Ignoring 2649 (21.0%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Ignoring 2614 (20.7%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Ignoring 2608 (20.6%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Ignoring 2586 (20.5%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Ignoring 2593 

In [11]:
from Evaluation.Evaluator import EvaluatorHoldout
evaluator_test = []
for u in urm_test:
    evaluator_test.append(EvaluatorHoldout(u, cutoff_list=[20], ignore_users=[]))

EvaluatorHoldout: Ignoring 2146 (17.0%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Ignoring 2177 (17.2%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Ignoring 2157 (17.1%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Ignoring 2132 (16.9%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Ignoring 2149 (17.0%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Ignoring 2129 (16.8%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Ignoring 2228 (17.6%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Ignoring 2183 (17.3%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Ignoring 2140 

In [12]:
from Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender
from Recommenders.KNN.ItemKNNCFRecommender import ItemKNNCFRecommender

recommenderClass = RP3betaRecommender



In [13]:
import optuna

# Replace the following with your MySQL database connection details
host = "db-mysql-nyc3-14396-do-user-15539286-0.c.db.ondigitalocean.com"
port = 25060
database_name = "recsys_optuna"
username = "doadmin"
password = "AVNS_EIQ1D-GbDGJWZPsN3Mt"

# Create an SQLAlchemy engine
mysql_url = f"mysql+pymysql://{username}:{password}@{host}:{port}/{database_name}"
storage = optuna.storages.RDBStorage(url=mysql_url)

In [14]:
study = optuna.create_study(study_name="RP3_XG_RECALL", direction="maximize", storage=storage, load_if_exists=True, sampler=optuna.samplers.TPESampler())

[I 2024-01-10 10:04:16,981] A new study created in RDB with name: RP3_XG_RECALL


In [15]:
def objective(trial):

    params = {
        'topK': trial.suggest_int('topK', 5, 1000),
        'alpha': trial.suggest_float('alpha', 0, 2),
        'beta': trial.suggest_float('beta', 0, 2),
    }

    RECALL = 0
    for i in range(k_fold):
        recommender = recommenderClass(urm_train[i])
        recommender.fit(**params)

        result_dict, _ = evaluator_validation[i].evaluateRecommender(recommender)

        RECALL += result_dict[metric_to_optimize].item()
    
    RECALL /= k_fold
    return RECALL

In [16]:
study.optimize(objective, n_trials=200)

RP3betaRecommender: URM Detected 508 ( 4.0%) users with no interactions.
RP3betaRecommender: URM Detected 332 ( 1.5%) items with no interactions.
RP3betaRecommender: Similarity column 22222 (100.0%), 2826.17 column/sec. Elapsed time 7.86 sec
EvaluatorHoldout: Processed 10057 (100.0%) in 4.86 sec. Users per second: 2071
RP3betaRecommender: URM Detected 440 ( 3.5%) users with no interactions.
RP3betaRecommender: URM Detected 325 ( 1.5%) items with no interactions.
RP3betaRecommender: Similarity column 22222 (100.0%), 2607.40 column/sec. Elapsed time 8.52 sec
EvaluatorHoldout: Processed 10040 (100.0%) in 5.17 sec. Users per second: 1943
RP3betaRecommender: URM Detected 479 ( 3.8%) users with no interactions.
RP3betaRecommender: URM Detected 345 ( 1.6%) items with no interactions.
RP3betaRecommender: Similarity column 22222 (100.0%), 2755.32 column/sec. Elapsed time 8.07 sec
EvaluatorHoldout: Processed 10017 (100.0%) in 5.40 sec. Users per second: 1856
RP3betaRecommender: URM Detected 451 

[I 2024-01-10 10:07:21,068] Trial 0 finished with value: 0.005161525552501001 and parameters: {'topK': 453, 'alpha': 1.2768241237474842, 'beta': 1.5559695795880208}. Best is trial 0 with value: 0.005161525552501001.


RP3betaRecommender: URM Detected 508 ( 4.0%) users with no interactions.
RP3betaRecommender: URM Detected 332 ( 1.5%) items with no interactions.
RP3betaRecommender: Similarity column 22222 (100.0%), 2128.69 column/sec. Elapsed time 10.44 sec
EvaluatorHoldout: Processed 10057 (100.0%) in 5.09 sec. Users per second: 1976
RP3betaRecommender: URM Detected 440 ( 3.5%) users with no interactions.
RP3betaRecommender: URM Detected 325 ( 1.5%) items with no interactions.
RP3betaRecommender: Similarity column 22222 (100.0%), 2187.50 column/sec. Elapsed time 10.16 sec
EvaluatorHoldout: Processed 10040 (100.0%) in 6.31 sec. Users per second: 1590
RP3betaRecommender: URM Detected 479 ( 3.8%) users with no interactions.
RP3betaRecommender: URM Detected 345 ( 1.6%) items with no interactions.
RP3betaRecommender: Similarity column 22222 (100.0%), 2217.57 column/sec. Elapsed time 10.02 sec
EvaluatorHoldout: Processed 10017 (100.0%) in 4.73 sec. Users per second: 2120
RP3betaRecommender: URM Detected 4

[I 2024-01-10 10:11:21,478] Trial 1 finished with value: 0.00384150420332834 and parameters: {'topK': 751, 'alpha': 1.1668810801332496, 'beta': 1.979243513628632}. Best is trial 0 with value: 0.005161525552501001.


RP3betaRecommender: URM Detected 508 ( 4.0%) users with no interactions.
RP3betaRecommender: URM Detected 332 ( 1.5%) items with no interactions.
RP3betaRecommender: Similarity column 22222 (100.0%), 3431.74 column/sec. Elapsed time 6.48 sec
EvaluatorHoldout: Processed 10057 (100.0%) in 5.50 sec. Users per second: 1830
RP3betaRecommender: URM Detected 440 ( 3.5%) users with no interactions.
RP3betaRecommender: URM Detected 325 ( 1.5%) items with no interactions.
RP3betaRecommender: Similarity column 22222 (100.0%), 2631.52 column/sec. Elapsed time 8.44 sec
EvaluatorHoldout: Processed 10040 (100.0%) in 5.56 sec. Users per second: 1805
RP3betaRecommender: URM Detected 479 ( 3.8%) users with no interactions.
RP3betaRecommender: URM Detected 345 ( 1.6%) items with no interactions.
RP3betaRecommender: Similarity column 22222 (100.0%), 2485.04 column/sec. Elapsed time 8.94 sec
EvaluatorHoldout: Processed 10017 (100.0%) in 4.36 sec. Users per second: 2295
RP3betaRecommender: URM Detected 451 

...

RP3betaRecommender: URM Detected 508 ( 4.0%) users with no interactions.
RP3betaRecommender: URM Detected 332 ( 1.5%) items with no interactions.
RP3betaRecommender: Similarity column 22222 (100.0%), 4539.25 column/sec. Elapsed time 4.90 sec
EvaluatorHoldout: Processed 10057 (100.0%) in 3.95 sec. Users per second: 2545
RP3betaRecommender: URM Detected 440 ( 3.5%) users with no interactions.
RP3betaRecommender: URM Detected 325 ( 1.5%) items with no interactions.
RP3betaRecommender: Similarity column 22222 (100.0%), 4906.50 column/sec. Elapsed time 4.53 sec
EvaluatorHoldout: Processed 10040 (100.0%) in 3.57 sec. Users per second: 2816
RP3betaRecommender: URM Detected 479 ( 3.8%) users with no interactions.
RP3betaRecommender: URM Detected 345 ( 1.6%) items with no interactions.
RP3betaRecommender: Similarity column 22222 (100.0%), 4855.05 column/sec. Elapsed time 4.58 sec
EvaluatorHoldout: Processed 10017 (100.0%) in 4.04 sec. Users per second: 2479
RP3betaRecommender: URM Detected 451 

[I 2024-01-10 14:13:03,084] Trial 246 finished with value: 0.16534334854633403 and parameters: {'topK': 118, 'alpha': 0.25254171914557005, 'beta': 0.21605574269222572}. Best is trial 208 with value: 0.16566850486147536.


RP3betaRecommender: URM Detected 508 ( 4.0%) users with no interactions.
RP3betaRecommender: URM Detected 332 ( 1.5%) items with no interactions.
RP3betaRecommender: Similarity column 22222 (100.0%), 4436.04 column/sec. Elapsed time 5.01 sec
EvaluatorHoldout: Processed 10057 (100.0%) in 4.38 sec. Users per second: 2295
RP3betaRecommender: URM Detected 440 ( 3.5%) users with no interactions.
RP3betaRecommender: URM Detected 325 ( 1.5%) items with no interactions.
RP3betaRecommender: Similarity column 22222 (100.0%), 4963.99 column/sec. Elapsed time 4.48 sec
EvaluatorHoldout: Processed 10040 (100.0%) in 3.04 sec. Users per second: 3297
RP3betaRecommender: URM Detected 479 ( 3.8%) users with no interactions.
RP3betaRecommender: URM Detected 345 ( 1.6%) items with no interactions.
RP3betaRecommender: Similarity column 22222 (100.0%), 6452.58 column/sec. Elapsed time 3.44 sec
EvaluatorHoldout: Processed 10017 (100.0%) in 4.18 sec. Users per second: 2394
RP3betaRecommender: URM Detected 451 

[W 2024-01-10 14:13:45,794] Trial 248 failed with parameters: {'topK': 126, 'alpha': 0.21984839203440418, 'beta': 0.18444780957693968} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/home/japo/miniconda3/envs/RecSysFramework/lib/python3.8/site-packages/optuna/study/_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
  File "/tmp/ipykernel_390153/1669587347.py", line 12, in objective
    recommender.fit(**params)
  File "/home/japo/RecSys-Challenge/Recommenders/GraphBased/RP3betaRecommender.py", line 94, in fit
    similarity_block = similarity_block.toarray()
  File "/home/japo/miniconda3/envs/RecSysFramework/lib/python3.8/site-packages/scipy/sparse/compressed.py", line 1042, in toarray
    csr_todense(M, N, x.indptr, x.indices, x.data, y)
KeyboardInterrupt
[W 2024-01-10 14:13:45,795] Trial 248 failed with value None.


KeyboardInterrupt: 

In [17]:
best_MAP_test = 0

for i in range(k_fold):
    recommender = recommenderClass(urm_train[i])
    recommender.fit(**study.best_params)

    result_dict, _ = evaluator_test[i].evaluateRecommender(recommender)

    best_MAP_test += result_dict[metric_to_optimize].item()

best_MAP_test /= k_fold
best_MAP_test

RP3betaRecommender: URM Detected 508 ( 4.0%) users with no interactions.
RP3betaRecommender: URM Detected 332 ( 1.5%) items with no interactions.
RP3betaRecommender: Similarity column 22222 (100.0%), 4587.92 column/sec. Elapsed time 4.84 sec
EvaluatorHoldout: Processed 10492 (100.0%) in 4.26 sec. Users per second: 2464
RP3betaRecommender: URM Detected 440 ( 3.5%) users with no interactions.
RP3betaRecommender: URM Detected 325 ( 1.5%) items with no interactions.
RP3betaRecommender: Similarity column 22222 (100.0%), 4778.21 column/sec. Elapsed time 4.65 sec
EvaluatorHoldout: Processed 10461 (100.0%) in 4.01 sec. Users per second: 2606
RP3betaRecommender: URM Detected 479 ( 3.8%) users with no interactions.
RP3betaRecommender: URM Detected 345 ( 1.6%) items with no interactions.
RP3betaRecommender: Similarity column 22222 (100.0%), 4609.23 column/sec. Elapsed time 4.82 sec
EvaluatorHoldout: Processed 10481 (100.0%) in 3.34 sec. Users per second: 3137
RP3betaRecommender: URM Detected 451 

0.16603511884374528

In [18]:
best_MAP_test = 0

for i in range(k_fold):
    recommender = recommenderClass(urm_train_validation[i])
    recommender.fit(**study.best_params)

    result_dict, _ = evaluator_test[i].evaluateRecommender(recommender)

    best_MAP_test += result_dict[metric_to_optimize].item()

best_MAP_test /= k_fold
best_MAP_test

RP3betaRecommender: URM Detected 249 ( 2.0%) users with no interactions.
RP3betaRecommender: URM Detected 113 ( 0.5%) items with no interactions.
RP3betaRecommender: Similarity column 22222 (100.0%), 4378.08 column/sec. Elapsed time 5.08 sec
EvaluatorHoldout: Processed 10492 (100.0%) in 4.57 sec. Users per second: 2295
RP3betaRecommender: URM Detected 191 ( 1.5%) users with no interactions.
RP3betaRecommender: URM Detected 101 ( 0.5%) items with no interactions.
RP3betaRecommender: Similarity column 22222 (100.0%), 4028.97 column/sec. Elapsed time 5.52 sec
EvaluatorHoldout: Processed 10461 (100.0%) in 5.17 sec. Users per second: 2024
RP3betaRecommender: URM Detected 235 ( 1.9%) users with no interactions.
RP3betaRecommender: URM Detected 105 ( 0.5%) items with no interactions.
RP3betaRecommender: Similarity column 22222 (100.0%), 4204.26 column/sec. Elapsed time 5.29 sec
EvaluatorHoldout: Processed 10481 (100.0%) in 3.58 sec. Users per second: 2929
RP3betaRecommender: URM Detected 218 

Exception during reset or similar
Traceback (most recent call last):
  File "/home/japo/miniconda3/envs/RecSysFramework/lib/python3.8/site-packages/sqlalchemy/pool/base.py", line 739, in _finalize_fairy
    fairy._reset(pool)
  File "/home/japo/miniconda3/envs/RecSysFramework/lib/python3.8/site-packages/sqlalchemy/pool/base.py", line 988, in _reset
    pool._dialect.do_rollback(self)
  File "/home/japo/miniconda3/envs/RecSysFramework/lib/python3.8/site-packages/sqlalchemy/engine/default.py", line 682, in do_rollback
    dbapi_connection.rollback()
  File "/home/japo/miniconda3/envs/RecSysFramework/lib/python3.8/site-packages/pymysql/connections.py", line 488, in rollback
    self._read_ok_packet()
  File "/home/japo/miniconda3/envs/RecSysFramework/lib/python3.8/site-packages/pymysql/connections.py", line 448, in _read_ok_packet
    pkt = self._read_packet()
  File "/home/japo/miniconda3/envs/RecSysFramework/lib/python3.8/site-packages/pymysql/connections.py", line 739, in _read_packet


KeyboardInterrupt: 